# Text summarization with small files with Anthropic Claude

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview

In this example, you are going to ingest a small amount of data (String data) directly into Amazon Bedrock API (using Anthropic Claude model) and give it an instruction to summarize the respective text.

### Architecture

![](./images/41-text-simple-1.png)

In this architecture:

1. A small piece of text (or small file) is loaded
1. A foundation model processes the input data
1. Model returns a response with the summary of the ingested text

### Use case

This approach can be used to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

### Challenges

This approach can be used when the input text or file fits within the model context length. In notebook `02.long-text-summarization-titan.ipynb`, we will explore an approach to address the challenge when users have large document(s) that exceed the token limit.

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [2]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Summarizing a short text with boto3
 
To learn detail of API request to Amazon Bedrock, this notebook introduces how to create API request and send the request via Boto3 rather than relying on langchain, which gives simpler API by wrapping Boto3 operation. 

### Request Syntax of InvokeModel in Boto3


We use `InvokeModel` API for sending request to a foundation model. Here is an example of API request for sending text to Anthropic Claude. Inference parameters in `textGenerationConfig` depends on the model that you are about to use. Inference paramerters of Anthropic Claude are:

- **temperature** tunes the degree of randomness in generation. Lower temperatures mean less random generations.
- **top_p** less than one keeps only the smallest set of most probable tokens with probabilities that add up to top_p or higher for generation.
- **top_k** can be used to reduce repetitiveness of generated tokens. The higher the value, the stronger a penalty is applied to previously present tokens, proportional to how many times they have already appeared in the prompt or prior generation.
- **max_tokens_to_sample** is maximum number of tokens to generate. Responses are not guaranteed to fill up to the maximum desired length.
- **stop_sequences** are sequences where the API will stop generating further tokens. The returned text will not contain the stop sequence.

```python
response = bedrock.invoke_model(body=
                                {"prompt":"this is where you place your input text",
                                 "max_tokens_to_sample":4096,
                                 "temperature":0.5,
                                 "top_k":250,
                                 "top_p":0.5,
                                 "stop_sequences":[]
                                },
                                modelId="anthropic.claude-v2", 
                                accept=accept, 
                                contentType=contentType)

```

### Writing prompt with text to be summarized

In this notebook, you can use any short text whose tokens are less than the maximum token of a foundation model. As an exmple of short text, let's take one paragraph of an [AWS blog post](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) about announcement of Amazon Bedrock.

The prompt starts with an instruction `Please provide a summary of the following text.`. 

In [3]:
prompt = """

Human: Please provide a summary of the following text.
<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

Assistant:"""

## Creating request body with prompt and inference parameters 

Following the request syntax of `invoke_model`, you create request body with the above prompt and inference parameters.

In [4]:
body = json.dumps({"prompt": prompt,
                 "max_tokens_to_sample":4096,
                 "temperature":0.5,
                 "top_k":250,
                 "top_p":0.5,
                 "stop_sequences":[]
                  }) 

## Invoke foundation model via Boto3

Here sends the API request to Amazon Bedrock with specifying request parameters `modelId`, `accept`, and `contentType`. Following the prompt, the foundation model in Amazon Bedrock summarizes the text.

In [5]:
modelId = 'anthropic.claude-v2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('completion'))

 Here is a summary of the key points from the text:

- AWS announced Amazon Bedrock, a new service that provides API access to large language models
(LLMs) from AI21 Labs, Anthropic, Stability AI, and Amazon.

- Bedrock aims to make generative AI more accessible to developers by providing easy access to
powerful text and image generation models.

- Bedrock includes access to Amazon's new Titan LLMs, as well as other third-party models.

- Bedrock is a scalable, reliable, and secure AWS managed service with a serverless experience.

- Developers can quickly get started, customize models with their own data, and integrate the models
into their applications using familiar AWS tools and capabilities.

- Bedrock removes the need to manage infrastructure for deploying generative AI models.


In the above the Bedrock service generates the entire summary for the given prompt in a single output, this can be slow if the output contains large amount of tokens. 

Below we explore the option how we can use Bedrock to stream the output such that the user could start consuming it as it is being generated by the model. For this Bedrock supports `invoke_model_with_response_stream` API providing `ResponseStream` that streams the output in form of chunks.

In [6]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"completion":" Here","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" is a summary of the key points from the text:\\n\\n- AWS announced Amazon Bedrock,","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" a new service that provides API access to large language","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" models (LLMs) from AI21 Labs, Anthropic,","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" Stability AI, and Amazon. \\n\\n- Bedrock aims to make gener","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":"ative AI more accessible to developers by allowing them","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" to easily integrate and deploy powerful text and","stop_reason":null,"stop":null}'}},
 {'chunk': {'bytes': b'{"completion":" image generation models into their applications.\\n\\n- Bedrock offers","stop_

Instead of generating the entire output, Bedrock sends smaller chunks from the model. This can be displayed in a consumable manner as well.

In [7]:
from IPython.display import display_markdown,Markdown,clear_output

In [8]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

 Here is a summary of the key points from the text:

- AWS announced Amazon Bedrock, a new service that provides API access to foundation models (FMs) from AI21 Labs, Anthropic, Stability AI, and Amazon. 

- Bedrock aims to make generative AI more accessible to builders by providing easy access to powerful text and image FMs.

- Bedrock includes access to Amazon's new Titan FMs - two large language models also announced. 

- Bedrock offers a serverless experience so customers can easily find, customize, and integrate FMs into their applications without managing infrastructure. 

- Bedrock integrates with Amazon SageMaker to help test, manage, and scale FMs.

- Overall, Bedrock seeks to democratize access to generative AI for all builders.

## Conclusion
You have now experimented with using `boto3` SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use case of generating a summary of AWS news about Amazon Bedrock in 2 different ways: entire output and streaming output generation.

### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Amazon Titan and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You